In [50]:
import os, cv2, re, random
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img
from keras import layers, models, optimizers
from keras import backend as K
from sklearn.model_selection import train_test_split

In [51]:
img_width = 150
img_height = 150
TRAIN_DIR = 'input/train/train'
TEST_DIR = 'input/test/test'
train_images_dogs_cats = [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR)] # use this for full dataset
test_images_dogs_cats = [TEST_DIR+i for i in os.listdir(TEST_DIR)]

In [52]:
# Helper function to sort the image files based on the numeric value in each file name.

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]

In [53]:
#Sort the traning set. Use 1300 images each of cats and dogs instead of all 25000 to speed up the learning process.

train_images_dogs_cats.sort(key=natural_keys)
train_images_dogs_cats = train_images_dogs_cats[0:1300] + train_images_dogs_cats[12500:13800] 

test_images_dogs_cats.sort(key=natural_keys)

In [54]:
# Now the images have to be represented in numbers. For this, using the openCV library read and resize the image.

# Generate labels for the supervised learning set.

# Below is the helper function to do so.

def prepare_data(list_of_images):
    """
    Returns two arrays: 
        x is an array of resized images
        y is an array of labels
    """
    x = [] # images as arrays
    y = [] # labels
    
    for image in list_of_images:
        x.append(cv2.resize(cv2.imread(image), (img_width,img_height), interpolation=cv2.INTER_CUBIC))
    
    for i in list_of_images:
        if 'dog' in i:
            y.append(1)
        elif 'cat' in i:
            y.append(0)
        #else:
            #print('neither cat nor dog name present in images')
            
    return x, y

In [55]:
#Generate X and Y using the helper function above

#Since K.image_data_format() is channel_last, input_shape to the first keras layer 
#will be (img_width, img_height,3). '3' since it is a color image

X, Y = prepare_data(train_images_dogs_cats)
print(K.image_data_format())

error: OpenCV(4.0.0) /Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/resize.cpp:3784: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [34]:
# First split the data in two sets, 80% for training, 20% for Val/Test)
X_train, X_val, Y_train, Y_val = train_test_split(X,Y, test_size=0.2, random_state=1)

In [35]:
nb_train_samples = len(X_train)
nb_validation_samples = len(X_val)
batch_size = 16
print(nb_train_samples)

0


In [26]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(150, 150,3)),
  tf.keras.layers.Dense(64, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
])
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, 
          Y_train, 
          epochs=5,
          batch_size=batch_size,
          steps_per_epoch=nb_train_samples//batch_size,
          validation_steps=nb_validation_samples //batch_size,
          shuffle=False
         )
model.summary

ValueError: When using data tensors as input to a model, you should specify the `steps_per_epoch` argument.

In [ ]:
model.evaluate(X_val, Y_val)